In [0]:
#Loading Required Libraries

import numpy as np
import pandas as pd
import pandas_profiling
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import lightgbm as lgb

In [0]:
#Loading the Dataset

train = pd.read_csv('/content/train_LZdllcl.csv')
test = pd.read_csv('/content/test_2umaH9m.csv')
sample = pd.read_csv('/content/sample_submission_M0L0uXE.csv')

In [0]:
pandas_profiling.ProfileReport(train)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,14
Number of observations,54808
Total Missing (%),0.9%
Total size in memory,5.9 MiB
Average record size in memory,112.0 B
Numeric,6
Categorical,5
Boolean,3
Date,0
Text (Unique),0
Rejected,0


In [0]:
df = train[train['is_promoted'] == 1]
pandas_profiling.ProfileReport(df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,15
Number of observations,4668
Total Missing (%),0.7%
Total size in memory,547.2 KiB
Average record size in memory,120.0 B
Numeric,7
Categorical,5
Boolean,2
Date,0
Text (Unique),0
Rejected,1


### Preprocessing

In [0]:
# Check the types of Feature
train.dtypes

employee_id               int64
department               object
region                   object
education                object
gender                   object
recruitment_channel      object
no_of_trainings           int64
age                       int64
previous_year_rating    float64
length_of_service         int64
KPIs_met >80%             int64
awards_won?               int64
avg_training_score        int64
is_promoted               int64
dtype: object

In [0]:
# Droping Region and Employee Id as these contain many unique values

train.drop(['region','employee_id'],axis=1,inplace = True)
test.drop(['region','employee_id'],axis=1,inplace = True)

In [0]:
# Imputing all NA values with 'A' for further imputation need

train = train.fillna('A')
test = test.fillna('A')

In [0]:
train[train['education']=='A'].head()

,department,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
10,Technology,A,m,sourcing,1,30,A,1,0,0,77,0
21,Operations,A,m,sourcing,1,41,4,11,0,0,57,0
32,Sales & Marketing,A,f,sourcing,1,24,1,2,0,0,48,0
43,Sales & Marketing,A,m,other,3,24,2,2,0,0,48,0
82,Sales & Marketing,A,m,sourcing,2,25,3,2,0,0,53,0


In [0]:
#Converting Categorical column to numerical for modeling

lst = ['department','gender','education','recruitment_channel']
for i in lst:
  print(i)
  le = preprocessing.LabelEncoder()
  le.fit(train[i])
  train[i] = le.transform(train[i])
  test[i] = le.transform(test[i])

department
gender
education
recruitment_channel


In [0]:
# Making earlier imputed values 250 so we can apply good imputation method

train.loc[train.previous_year_rating == 'A', 'previous_year_rating'] = 250.0
test.loc[test.previous_year_rating == 'A', 'previous_year_rating'] = 250.0
train.loc[train.education == 0, 'education'] = 250
test.loc[test.education == 0, 'education'] = 250
print(train['previous_year_rating'].unique())
print(train['education'].unique())

[  5   3   1   4 250   2]
[  3   1 250   2]


In [0]:
# spliting data into feature and target
 
y = train['is_promoted']
train.drop(['is_promoted'],axis=1,inplace=True)

In [0]:
train.dtypes

department              int64
education               int64
gender                  int64
recruitment_channel     int64
no_of_trainings         int64
age                     int64
previous_year_rating    int64
length_of_service       int64
KPIs_met >80%           int64
awards_won?             int64
avg_training_score      int64
dtype: object

In [0]:
# Imputing with iterative imputer
imp = IterativeImputer(max_iter=10, random_state=0,missing_values=250)
imp.fit(train)
train = imp.transform(train)
test = imp.transform(test)

In [0]:
# Featureengineering using polynomial feature

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(4)
train = poly.fit_transform(train)
test = poly.fit_transform(test)

In [0]:
# scaling the features
scaler = StandardScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

In [39]:
# dividing Dataset into train test split using stratify sampling

X_train, X_test,y_train,y_test = train_test_split(train,y,stratify=y)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(41106, 1365)
(41106,)
(13702, 1365)
(13702,)


In [0]:
# Modeling with LightGBM

import lightgbm as lgb
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.05
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.1
params['num_leaves'] = 15
params['min_data'] = 128
params['max_depth'] = 8
clf = lgb.train(params, d_train, 150)

In [0]:
#Finding Probablisitic prediction about data
pred_prob1 = clf.predict(X_train)
pred_prob2 = clf.predict(X_test)

In [73]:
#Getting optimum threshold based on f1 score

pred1 = np.zeros(len(pred_prob1))
pred2 = np.zeros(len(pred_prob2))
aaa = 0
bbb = 0
for i in range(1,100):
  pred1[pred_prob1 > (i/100)] = 1
  pred1[pred_prob1 < (i/100)] = 0
  pred2[pred_prob2 > (i/100)] = 1
  pred2[pred_prob2 < (i/100)] = 0
  sc1 = f1_score(y_train, pred1, average='macro')
  sc2 = f1_score(y_test, pred2, average='macro')
  if sc1 > aaa:
    aaa = sc1
    th1 = i/100
  if sc2 >bbb:
    bbb = sc2
    th2 = i/100
print('Threshold', th1,th2)

Threshold 0.24 0.27


In [0]:

th1 = (th1+th2)/2

In [76]:
# making prediction from thresholding

pred1[pred_prob1 > th1] = 1
pred1[pred_prob1 < th1] = 0
pred2[pred_prob2 > th1] = 1
pred2[pred_prob2 < th1] = 0
print('------------TRAIN--------------')
print(f1_score(y_train, pred1, average='macro'))
print(f1_score(y_train, pred1, average='micro'))
print(f1_score(y_train, pred1, average='weighted'))
print('------------Test--------------')
print(f1_score(y_test, pred2, average='macro'))
print(f1_score(y_test, pred2, average='micro'))
print(f1_score(y_test, pred2, average='weighted'))

------------TRAIN--------------
0.7742609471856471
0.9415657081691238
0.9354953082283872
------------Test--------------
0.7404081627894369
0.9335863377609108
0.9261991520513418


In [77]:
#Computing test and train AUC

print('Test AUC:',roc_auc_score(y_test,pred_prob2))
print('Train AUC:',roc_auc_score(y_train,pred_prob1))

Test AUC: 0.9040280018006138
Train AUC: 0.9333523451293438


In [0]:
# Once model parameter is tuned train the model on complete data

import lightgbm as lgb
d_train = lgb.Dataset(train, label=y)
params = {}
params['learning_rate'] = 0.05
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.1
params['num_leaves'] = 15
params['min_data'] = 128
params['max_depth'] = 8
clf = lgb.train(params, d_train, 150)

In [0]:
#making Prediction based on above find threshold

pred_prob = clf.predict(test)
pred = np.zeros(len(pred_prob),dtype=int)
pred[pred_prob > th1] = 1
pred[pred_prob < th1] = 0

In [0]:
#saving output in a file

aa= pd.read_csv('/content/test_2umaH9m.csv')
aa = aa['employee_id']
aa = pd.DataFrame(aa)
aa['is_promoted'] = pred
aa.to_csv('output1.csv',index = False)